<a href="https://colab.research.google.com/github/fricova/lse_network_analysis/blob/main/FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras # define and train NN
import sklearn # train test split
import pandas as pd
import numpy as np # matrix multiplication
from sklearn.preprocessing import MinMaxScaler # MinMaxScaler to scale everything [0,1]
import matplotlib.pyplot as plt # plotting


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/

'Colab Notebooks'		 data_for_training_without_weekend.csv
'Copy of coronatweets.ipynb'	'Glemping Trip'
 corona_tweets_101-200.zip	'PB434 Seminar Prezi.gslides'
 corona_tweets_1-100.zip	 PRICES_NO_WEEKENDS.csv
 corona_tweets_theta-omega.zip


In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/PRICES_NO_WEEKENDS.csv")
data

,Unnamed: 0,date,anticipation,fear,joy,trust,syuzhet_index,bing_index,nrc_index,bing_index_norm,nrc_index_norm,syuzhet_index_norm,SP500,fear_norm,joy_norm,trust_norm,anticipation_norm,syuzhet_index_z,fear_z,joy_z,anticipation_z,trust_z,nrc_index_z,bing_index_z,weekday
0,2,2020-02-28,0.485801,0.703907,0.281224,0.602122,-0.425539,-0.599669,-0.346609,-0.800205,-0.651246,-0.695493,2954.22,0.793227,-0.113220,-0.037947,0.634220,-3.174923,2.672333,0.430044,1.928683,0.465933,-2.886255,-3.043472,5
1,5,2020-03-02,0.436957,0.617359,0.221443,0.632146,-0.324243,-0.498966,-0.147359,-0.566565,-0.106651,-0.370656,3090.23,0.328403,-0.886903,0.077953,0.282165,-2.175104,1.275395,-1.976391,0.492236,1.114476,-1.329017,-2.315678,1
2,6,2020-03-03,0.473808,0.674835,0.247063,0.583301,-0.336651,-0.503119,-0.244285,-0.576201,-0.371573,-0.410445,3003.37,0.637090,-0.555337,-0.110602,0.547775,-2.297570,2.203092,-0.945099,1.575972,0.059378,-2.086544,-2.345693,2
3,7,2020-03-04,0.475025,0.643650,0.251432,0.569121,-0.314615,-0.510885,-0.229937,-0.594217,-0.332357,-0.339779,3130.12,0.469602,-0.498784,-0.165340,0.556547,-2.080067,1.699740,-0.769200,1.611764,-0.246916,-1.974410,-2.401815,3
4,8,2020-03-05,0.479278,0.649734,0.248655,0.575449,-0.319109,-0.512931,-0.265848,-0.598964,-0.430508,-0.354191,3023.94,0.502277,-0.534732,-0.140911,0.587202,-2.124428,1.797937,-0.881010,1.736844,-0.110221,-2.255066,-2.416603,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,216,2020-09-29,0.442886,0.561427,0.270502,0.586078,-0.071379,-0.090919,0.075395,0.380138,0.502184,0.440235,3335.47,0.028009,-0.251989,-0.099882,0.324902,0.320745,0.372619,-0.001579,0.666608,0.119364,0.411912,0.633319,2
149,217,2020-09-30,0.423321,0.565903,0.272983,0.610541,-0.095622,0.001158,0.099206,0.593766,0.567266,0.362491,3363.00,0.052049,-0.219881,-0.005446,0.183885,0.081457,0.444866,0.098289,0.091234,0.647802,0.598008,1.298775,3
150,218,2020-10-01,0.438175,0.539686,0.298725,0.589078,-0.003546,-0.058103,0.145464,0.456275,0.693699,0.657762,3380.80,-0.088755,0.113271,-0.088301,0.290947,0.990272,0.021706,1.134513,0.528069,0.184171,0.959536,0.870487,4
151,219,2020-10-02,0.453991,0.447291,0.322588,0.680911,-0.006623,0.176253,0.156574,1.000000,0.724066,0.647895,3348.44,-0.584981,0.422097,0.266203,0.404939,0.959904,-1.469604,2.095075,0.993175,2.167869,1.046368,2.564202,5


In [ ]:
from keras import layers # modules 
from keras import regularizers

In [ ]:
sp500 = data['SP500']
sp500_mean = np.mean(sp500.values)
sp500_std = np.std(sp500.values)

In [ ]:
def direction_accuracy(x, y, model):
  real_difference = y - x[:, -1]
  real_direction = np.sign(real_difference)
  predicted_difference = model.predict(x) - x[:, -1]
  predicted_direction = np.sign(predicted_difference)
  return (real_direction == predicted_direction).mean()

In [ ]:
def create_model(n_of_inputs):
  input = keras.Input(shape=(n_of_inputs))
  x = layers.Dense(32)(input) # the first layer is dense/fully connected, not sparse - normal NN layer
  x = layers.LeakyReLU()(x)  # the first activation function is a leaky ReLu, because just 0 would make the NN quite difficult to train
  x = layers.Dense(16)(x) # the second layer
  x = layers.LeakyReLU()(x)
  x = layers.Dense(1)(x) # the last layer outputs just one thing 

  model = keras.Model(inputs=input, outputs=x)
  # model.summary()

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_percentage_error']) # we use adam optimizer, this means how you change the weights given the derivative w.r.t. the weight
  return model

In [ ]:
from sklearn.preprocessing import StandardScaler
def get_data_for_columns(d, columns, lag, k, k_index):
  d = d.copy()
  length_of_1_k = len(d) // k
  d_train_first = d[0:length_of_1_k * k_index]
  d_test = d[length_of_1_k * k_index:(length_of_1_k * k_index) + length_of_1_k]
  d_train_second = d[(length_of_1_k * k_index) + length_of_1_k:-1]
  if len(columns) > 0:
    d[columns] = MinMaxScaler().fit_transform(d[columns]) # normalizing [0,1]
  X_train = []
  y_train = []
  X_test = []
  y_test = []
  for i in range(0, len(d_train_first) - lag - 1):
    emotions = d_train_first[i:i+(lag)][columns] # 3 columns for each emotion
    sp500 = d_train_first[i:i+(lag+1)]['SP500'].values
    # sp500 = (sp500 - sp500_mean) / sp500_std
    # sp500_differences = np.diff(sp500.values)
    X_train.append(np.concatenate((emotions.values.flatten(), sp500[:-1]))) # we join all emotion lists together + SP500 as last columns 
    y_train.append(sp500[-1]) # we are predicting last value of SP500
  for i in range(0, len(d_train_second) - lag - 1):
    emotions = d_train_second[i:i+(lag)][columns] # 3 columns for each emotion
    sp500 = d_train_second[i:i+(lag+1)]['SP500'].values
    # sp500 = (sp500 - sp500_mean) / sp500_std
    # sp500_differences = np.diff(sp500.values)
    X_train.append(np.concatenate((emotions.values.flatten(), sp500[:-1])))
    y_train.append(sp500[-1]) # we are predicting last value of SP500
  for i in range(0, len(d_test) - lag - 1):
    emotions = d_test[i:i+(lag)][columns]
    sp500 = d_test[i:i+(lag+1)]['SP500'].values # we do the same for test set
    # sp500 = (sp500 - sp500_mean) / sp500_std
    # sp500_differences = np.diff(sp500.values)

    X_test.append(np.concatenate((emotions.values.flatten(), sp500[:-1])))

    y_test.append(sp500[-1])
  X_train = np.vstack(X_train) # we stack all training vectors together
  y_train = np.array(y_train)
  X_test = np.vstack(X_test) # we stack all test vectors together
  y_test = np.array(y_test)
  return X_train, X_test, y_train, y_test 

def train_model_with_columns(d, columns, lag, n_of_trials):
  mapes = []
  predicteds = []
  actuals = []
  dir_accs = []
  for i in range(n_of_trials):
    X_train, X_test, y_train, y_test = get_data_for_columns(d, columns, lag, n_of_trials, i)
    model = create_model(X_train.shape[1])
    model.fit(X_train, y_train, batch_size=10, epochs=10, verbose=0)
    mapes.append(model.evaluate(X_test, y_test)[1])
    predicteds.append(model.predict(X_test).flatten())
    actuals.append(y_test)
    dir_accs.append(direction_accuracy(X_test, y_test, model))
  return mapes, predicteds, actuals, dir_accs
  # we train the same model multiple times, average mape 

In [ ]:
k = 10 # the number of folds in the k-fold cross-validation is 10

In [ ]:
mape_without_columns = 100000000
best_lag = -1
predictes_without_columns = []
actuals_without_columns = []
direction_accs_without_columns = []
for lag in range(1, 7):
  columns_of_interest = [] # no emotion columns in the model, just SP500 to see if adding emotions improves forecasting accuracy

  curr_mapes_without_columns, curr_predictes_without_columns, curr_actuals_without_columns, curr_accs = \
    train_model_with_columns(data, columns_of_interest, lag, k)
  current_mape = np.array(curr_mapes_without_columns).mean()
  if current_mape < mape_without_columns:
    mape_without_columns = current_mape
    best_lag = lag
    predictes_without_columns = curr_predictes_without_columns
    actuals_without_columns = curr_actuals_without_columns
    direction_accs_without_columns = curr_accs

1/1 [==============================] - 0s 271ms/step - loss: 3387.4097 - mean_absolute_percentage_error: 1.5425


In [ ]:
inputs = [
  {"columns": ["fear"], "name": "fear" },
  {"columns": ["syuzhet_index"], "name": "index"},
  {"columns": ["fear", "anticipation"], "name": "fear_anticipation" },
  {"columns": ["fear", "joy"], "name": "fear_joy" },
  {"columns": ["fear", "trust"], "name": "fear_trust" },
  {"columns": ["fear", "joy", "anticipation"], "name": "fear_joy_anticipation" },
  {"columns": ["fear", "joy", "trust"], "name": "fear_joy_trust" },
  {"columns": ["fear", "joy", "anticipation","trust"], "name": "fear_joy_anticipation_trust" },
]

outputs = {}

for input in inputs:
  columns_of_interest = input["columns"]
  mapes, predictes, actuals, accs = train_model_with_columns(data, columns_of_interest, best_lag, k)
  mape = np.array(mapes).mean()
  outputs[input["name"]] = {"mape": mape, "predicteds": predictes, "actuals": actuals, "dir_accs": accs}

1/1 [==============================] - 0s 247ms/step - loss: 2361.9426 - mean_absolute_percentage_error: 1.2927


In [ ]:
outputs["no_columns"] = {
  "mape": mape_without_columns,
  "predicteds": predictes_without_columns,
  "actuals": actuals_without_columns,
  "dir_accs": direction_accs_without_columns
}

pd.DataFrame(outputs).T

,mape,predicteds,actuals,dir_accs
fear,1.64915,"[[2998.4097, 3084.356, 3045.6523, 3120.0474, 3...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.6111111111111112, 0.7291666666666666, 0.694..."
index,1.67993,"[[3005.8093, 3072.9053, 3052.4666, 3106.886, 3...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.625, 0.7222222222222222, 0.6944444444444444..."
fear_anticipation,1.7529,"[[3051.8484, 3051.8628, 3095.491, 3080.4194, 3...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.6388888888888888, 0.7291666666666666, 0.687..."
fear_joy,1.74408,"[[3000.5237, 3094.3665, 3048.2668, 3130.7449, ...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.5972222222222222, 0.7083333333333334, 0.666..."
fear_trust,1.74323,"[[3053.5486, 3037.826, 3096.5098, 3064.5942, 2...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.6319444444444444, 0.7083333333333334, 0.687..."
fear_joy_anticipation,1.74537,"[[3033.0828, 3056.1453, 3078.2205, 3086.1077, ...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.6319444444444444, 0.7083333333333334, 0.687..."
fear_joy_trust,1.7125,"[[2978.8682, 3093.0076, 3027.0288, 3131.2637, ...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.5972222222222222, 0.7083333333333334, 0.687..."
fear_joy_anticipation_trust,1.79462,"[[3047.6711, 3039.7834, 3091.146, 3067.2085, 2...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.6319444444444444, 0.7291666666666666, 0.680..."
no_columns,1.70837,"[[3022.9316, 3068.3594, 3068.3938, 3100.6118, ...","[[3003.37, 3130.12, 3023.94, 2972.37, 2746.56,...","[0.625, 0.7152777777777778, 0.6944444444444444..."


In [ ]:
dir_accs = pd.DataFrame(outputs).T["dir_accs"]
dir_accs.apply(np.mean)

fear                           0.650000
index                          0.636806
fear_anticipation              0.647917
fear_joy                       0.640278
fear_trust                     0.635417
fear_joy_anticipation          0.637500
fear_joy_trust                 0.655556
fear_joy_anticipation_trust    0.636111
no_columns                     0.638194
Name: dir_accs, dtype: float64

In [ ]:
best_lag

2